# Load Models

In [11]:
from keras.models import load_model
model = load_model('saved_models/model-9862.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 63, 63, 64)        18496     
__________

# Get the symbolic outputs of each "key" layer


In [8]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
print(layer_dict.keys())

dict_keys(['conv2d_1', 'activation_1', 'conv2d_2', 'activation_2', 'max_pooling2d_1', 'dropout_1', 'conv2d_3', 'activation_3', 'conv2d_4', 'activation_4', 'max_pooling2d_2', 'dropout_2', 'conv2d_5', 'activation_5', 'conv2d_6', 'activation_6', 'max_pooling2d_3', 'dropout_3', 'flatten_1', 'dense_1', 'activation_7', 'dropout_4', 'dense_2', 'activation_8'])


In [6]:
from keras import backend as K
from keras.preprocessing.image import img_to_array
import cv2

path = "data-sanitized/0000006.png"
img = cv2.imread(path)
img = img_to_array(img)
img.astype('float32')
img /= 255
input_img = img


layer_name = 'conv2d_1'
filter_index = 0  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
loss = K.mean(layer_output[:, :, :, filter_index])

# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, input_img)[0]

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([input_img], [loss, grads])

ValueError: Tried to convert 'x' to a tensor and failed. Error: None values not supported.